First, import needed modules

In [1]:
import numpy as np
import pandas as pd
import os

Initalize variables

In [2]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/code


Load Datasets

In [3]:
dataset = pd.read_csv(f'{path}/data/btw13_wbz/BTW13_Erststimmen_Wahlbezirke.csv', sep=';')

/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_19536/2016819026.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f'{path}/data/btw13_wbz/BTW13_Erststimmen_Wahlbezirke.csv', sep=';')
